In [1]:
# Make necessary imports
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.mllib.random import RandomRDDs
from pyspark.sql import SQLContext
from pyspark import sql
import numpy as np

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = sql.SQLContext(sc) #?

22/03/05 15:08:03 WARN Utils: Your hostname, Kamiels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 129.104.246.162 instead (on interface en0)
22/03/05 15:08:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 15:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/05 15:08:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/05 15:08:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
/Users/kamielfokkink/opt/anaconda3/envs/Databases/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warning

In [2]:
def normalization(x):
    s = sum(x[1])
    s_last = 0
    x_new = []
    for i in range(len(x[1])-1):
        x_new.append(x[1][i] / s)
        s_last += x_new[i]
    x_new.append(1 - s_last)
    return (x[0], np.array(x_new))

def create_uniform_rdd_rowindexed_matrix(sc, nrow, ncol, normalize=False):
    rdd = []
    for i in range(nrow):
        rdd.append((i, np.random.rand(ncol)))
    #print(rdd)
    rdd = sc.parallelize(rdd)
    if normalize:
        rdd = rdd.map(normalization)
    return rdd

In [4]:
H = create_uniform_rdd_rowindexed_matrix(sc, 4, 4, normalize=True)
H.collect()

[(0, array([0.34860894, 0.16783557, 0.35134763, 0.13220786])),
 (1, array([0.10822922, 0.3555388 , 0.32625561, 0.20997637])),
 (2, array([0.02984418, 0.29335671, 0.33616264, 0.34063647])),
 (3, array([0.0272727 , 0.07928486, 0.63217075, 0.2612717 ]))]

In [3]:
def update_h(hxy_tuple):
    h, x, y = hxy_tuple
    d = h * y 
    e = h * x
    #h_new = h * (x+d) / (y+e)
    h_new = h * np.divide(x+d, y+e)
    return h_new

In [4]:
class MRAlgorithm:
    
    def __init__(self, datapath, k):
        self.k = k
        self.n = 2649429
        self.m = 17770
        self.sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
        self.input = self.sc.textFile(datapath).map(lambda x: (int(x.split(',')[0])-1,
                                                               int(x.split(',')[1])-1,int(x.split(',')[2])))
        self.A = self.input
        self.H = create_uniform_rdd_rowindexed_matrix(self.sc, self.n, self.k, normalize=True)
        self.W = create_uniform_rdd_rowindexed_matrix(self.sc, self.m, self.k)
        
    def compute_H(self):
        print("Compute H")
    
        ## compute X
        # M1
        W_b = self.sc.broadcast(self.W.collect()) # a dictionary where the key is the index i
        Aw = self.A.map(lambda x: (x[1], x[2]*W_b.value[x[0]][1])) #Wi.T
        #print(Aw.collect())
        # R1 
        X = Aw.reduceByKey(lambda x, y: x+y) # add a combiner? #.map(lambda x: (x[0], x[1]))
    
        
        ## compute B
        # M2
        self.ww = self.W.map(lambda x: (None, x[1]@x[1].T)) # w.T.dot(w)? #import numpy.linalg
        # from pyspark.mllib.linalg.distributed import *
        # as_block_matrix(w.T).multiply(as_block_matrix(w))
        # https://stackoverflow.com/questions/37766213/spark-matrix-multiplication-with-python
        # R2
        self.B = self.ww.reduceByKey(lambda x, y: x+y).map(lambda x: x[1]) # without map?
        
        ## compute Y
        # M3
        self.B_b = self.sc.broadcast(self.B.collect())
        self.y = self.H.map(lambda x: (x[0], self.B_b.value@x[1]))
        
        ## update H
        # M4
        #self.hxj = self.H.join(self.y).join(self.X)
        self.hxj = self.H.join(self.y).join(X)
        # R4
        # why is it a reduce phase instead of a map phase?
        self.hnew = self.hxj.reduceByKey(lambda x: (x[0], update_h(x[1])))
        print(self.hnew)
        self.H = self.hnew
        
    def compute_W(self):
        print("Compute W")
        #M5: Compute matrix S (repartition join of H and A)
        tmp_M5 = self.H.join(self.A.map(lambda x: (x[1], (x[0], x[2]))))

        #R5: Emit each hj according to row i
        tmp_M5 = tmp_M5.map(lambda x: (x[1][1][0], x[1][1][1] * x[1][0]))

        
        
        #M6: Finish the computation of matrix S -- identity mapper so no code needed
        #R6: Emit each row in matrix S
        S = tmp_M5.reduceByKey(lambda x, y : x +y)

        
        #M7: Calculate matrix C (same as calculating B in compute_H)
        tmp_M7 = self.H.map(lambda x: (0, np.outer(x[1], x[1].T)))
        #R7: Emit matrix C from the sum of matrices
        C = tmp_M7.reduceByKey(lambda x, y : x +y).map(lambda x: x[1])
        
        #M8: Compute matrix T
        C_b = self.sc.broadcast(C.collect())
        T = self.W.map(lambda x: (x[0], x[1]@C_b.value))
        

        #M9: update W with broadcast join of S and T. W = W * S/T
        T_b = self.sc.broadcast(T.collectAsMap())
        S_b = self.sc.broadcast(S.collectAsMap())
        self.W = self.W.map(lambda x: (x[0], x[1]*np.divide(S_b.value[x[0]], T_b.value[x[0]])))

        
    def assign_clusters(self):
        #M10: Map user to cluster with highest probability
        #self.clusters = self.H.map(lambda x: (x[0], x[1].index(max(x[1]))))
        self.clusters = self.H.map(lambda x: (x[0], np.where(x[1] == max(x[1]))))
        
        #M11: Emit a 1 for each user that is in a cluster
        self.clustersizes = self.clusters.map(lambda x: (x[1], 1))
        
        #R11: Count the number of users per cluster
        self.clustersizes = self.clustersizes.reduceByKey(lambda x,y: x+y)
        
    def RM2_distribution(self):
        #M12: Emit each rating that a user gave
        self.ratings = self.input.map(lambda x: (x[1], x[2]))
        
        #R12: Sum ratings given by user
        self.ratings = self.ratings.reduceByKey(lambda x, y: x+y)
        
        #Accumulator containing total count of ratings
        globalcount = self.sc.accumulator(0)
        self.ratings.foreach(lambda x: globalcount.add(x[1]))
        
        #M13: Emit each rating that an item received
        self.items = self.input.map(lambda x: (x[0], x[2]))
        
        #Broadcast the total count of ratings, to be used in next reduce
        globalcount = self.sc.broadcast(globalcount.value)
        
        #R13: Compute the probability of item in the collection
        self.items = self.items.reduceByKey(lambda x,y: x+y)
        
        #Divide previous values by the total count of ratings, to normalize probabilities
        self.items = self.items.map(lambda x: (x[0], x[1]/globalcount.value))
        
        #Broadcast all users and which cluster they are in
        clusters = self.sc.broadcast(self.clusters.collectAsMap())
        
        #M14-a: Map each rating from input to the cluster assigned to user
        self.input2cluster = self.input.map(lambda x: (clusters[x[1]], (x[1], x[0], x[2])))
        
        #M14-b: Map sum of ratings of a user to their assigned cluster
        self.ratings2cluster = self.ratings.map(lambda x: (clusters[x[0]], (x[0], x[1])))
        
        #Repartition join these two maps by cluster
        self.fullClusters = self.input2cluster.join(self.ratings2cluster)
        
        #Broadcast sizes of the clusters
        clustersizes = self.sc.broadcast(self.clustersizes.collectAsMap())
        
        #Broadcast probabilities of each item
        items = self.sc.broadcast(self.items.collectAsMap())
        
        #Final reduce combines all information from above, needs W and H

In [ ]:
Algorithm = MRAlgorithm('../data/small_dataset.txt', k=5)
#Algorithm.compute_H()
Algorithm.compute_W()
Algorithm.assign_clusters()
Algorithm.RM2_distribution()

22/03/05 15:13:09 WARN Utils: Your hostname, Kamiels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 129.104.246.162 instead (on interface en0)
22/03/05 15:13:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/05 15:13:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/05 15:13:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/05 15:13:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [51]:
Algorithm.input.take(10)

[(1, 1488844, 3),
 (1, 822109, 5),
 (1, 885013, 4),
 (1, 30878, 4),
 (1, 823519, 3),
 (1, 893988, 3),
 (1, 124105, 4),
 (1, 1248029, 3),
 (1, 1842128, 4),
 (1, 2238063, 3)]

In [52]:
Algorithm.clusters.collectAsMap()

{1: 2, 2: 1, 3: 0, 4: 0}

In [53]:
Algorithm.clustersizes.collect()

[(0, 2), (1, 1), (2, 1)]

In [54]:
Algorithm.ratings.take(5)

[(1488844, 9), (30878, 8), (893988, 3), (1842128, 4), (2207774, 5)]

In [55]:
Algorithm.items.take(5)

[(2, 0.0031263442977540004),
 (4, 0.002356875836872686),
 (6, 0.019042829705117875),
 (8, 0.2881568503898842),
 (10, 0.004798574968645675)]